# Parteiprogramme als Datenquelle

In [ ]:
import pandas as pd
import re
import json
import seaborn as sns

from dotenv import load_dotenv
from pathlib import Path
from pandarallel import pandarallel
from pdfminer.high_level import extract_text
from studienarbeit.utils.cleaning import CleanText
from matplotlib import pyplot as plt
from textblob_de.sentiments import PatternAnalyzer
from collections import Counter
from nltk import ngrams
from nltk.tokenize import sent_tokenize

In [ ]:
load_dotenv()
sns.set(style="white", palette="muted", rc={"figure.figsize": (20, 8)})
pandarallel.initialize()

In [ ]:
FAST_MODE = True
data_dir = Path("../../data/party_programs")

## 1. Business Understanding

In unseren Untersuchungszeitraum von 2017 bis 2021, die 19. Wahlperiode des Deutschen Bundestages, fallen neben den Bundestagswahlen 2017 und 2021 die Europawahl 2019 sowie zahlreiche Landtagswahlen, zu denen die antretenden Parteien jeweils ein Wahlprogramm veröffentlichen. Diese Wahlprogramme sind für uns von Interesse, da sie umfangreiche Texte mit der jeweiligen politischen Prägung zu einer großen Bandbreite an Themen bereitstellen und damit viele Informationen dazu bieten, welche Themen von einer Partei immer wieder mit Priorität behandelt werden und welche Sprache dabei verwendet wird. Die Wahlprogramme können üblicherweise als PDF-Datei von der Webseite der jeweiligen Partei bezogen werden.

## 2. Beschaffung der Daten

In Summe stehen 82 Wahlprogramme von AfD, FDP, Grünen, Linken, SPD und CDU/CSU zur Verfügung.

In [ ]:
df_party_programs_overview = pd.read_csv(data_dir / "overview.csv", delimiter=";")
df_party_programs_overview

### Hinweis: Nicht parsbare Wahlprogramme

Der entwickelte Algorithmus zum Auslesen des Texten und parsen der einzelnen Paragraphen eines Wahlprogramms funktioniert für die meisten gegebenen Wahlprogramme im PDF-Format. Es gibt jedoch einige Ausnahmen, bei denen entweder der Text nicht erkannt werden kann oder die Formatierung so ist, dass der Algorithmus ohne erhebliche Anpassungen nicht funktioniert. Dies betrifft die folgenden Wahlprogramme:
- AfD + FDP LTW MV 21
- FDP + CDU LTW 21 Berlin
- AfD LTW 21 Sachsen-Anhalt
- Grüne LTW 21 Bade-Württemberg
- SPD LTW 21 Rheinland-Pfalz
- SPD + CDU LTW 20 Hamburg
- AfD LTW 19 Thüringen
- FDP LTW 19 Brandenburg
- AfD Europawahl 2019
- AfD + SPD LTW 18 Hessen
- AfD + Linke LTW 18 Bayern

In [ ]:
def initial_cleaning(text, lower=False, gender_symbols=["*", ":"]):
    if lower:
        text = text.lower()
    
    text = text.replace("-\n", "- ")
    text = re.sub("-\s+", "-", text)
    text = text.replace("-oder", "- oder")
    text = text.replace("-und", "- und")
    text = re.sub("([a-zßäöü])-([a-zßäöü])", r"\1\2", text)
    text = re.sub("[\u2022\u2023\u25E6\u2043\u2219\uf0b7\u25fc]\s", "", text)
    text = text.replace("\n", " ")
    text = re.sub("\s+", " ", text)
    text = text.strip()
    
    # Clean words with 'Genderstern'
    for symbol in gender_symbols:
        text = re.sub(f"([a-zßäöü])\{symbol}innen([a-zßäöü]?)", r"\1\2", text)
        text = re.sub(f"([a-zßäöü])\{symbol}in([a-zßäöü]?)", r"\1\2", text)
        text = text.replace(f"Sinti{symbol}zze und Rom{symbol}nja", "Sinti und Roma")
        text = text.replace(f"der{symbol}die", "der")
        text = text.replace(f"die{symbol}der", "der")
        text = text.replace(f"den{symbol}die", "den")
        text = text.replace(f"dem{symbol}der", "dem")
        text = text.replace(f"der{symbol}s", "des")
        text = text.replace(f"eines{symbol}einer", "eines")
        text = text.replace(f"einer{symbol}s", "eines")
        text = text.replace(f"ihre{symbol}seine", "seine")
        text = text.replace(f"seiner{symbol}ihrer", "seiner")
        text = text.replace(f"jeder{symbol}m", "jedem")
        text = text.replace(f"Sie{symbol}Er", "Er")
        text = text.replace(f"des{symbol}der", "des")
        text = text.replace(f"welchem{symbol}welcher", "welchem")
        text = text.replace(f"{symbol}r", "r")
        text = text.replace(f"{symbol}n", "n")
        text = text.replace(f"{symbol}e", "n")
        
    # Party specific cleaning
    # AfD
    text = text.replace("die AfD", "wir")
    text = text.replace("Die AfD", "Wir")
    
    # FDP
    text = text.replace("wir freien demokraten", "wir")
    text = text.replace("Freie Demokraten", "")
    
    # Greens
    text = re.sub("Bundestagswahlprogramm 2021(.*)Bundestagswahlprogramm 2021", "", text)
    text = re.sub("Bundestagswahlprogramm 2021BÜNDNIS 90 / DIE GRÜNENKapitel\s\d+", "", text)
    text = text.replace("Bundestagswahlprogramm 2021BÜNDNIS 90 / DIE GRÜNENEpilog", "")
    text = text.replace("Bundestagswahlprogramm 2021BÜNDNIS 90 / DIE GRÜNENEinleitung", "")
    text = text.replace("Bereit, weil Ihr es seid.", "")
    text = text.replace("bündnis 90/die grünen in sachsenlandtagswahlprogramm 2019", "")
    text = text.replace("wir grüne", "wir")
    text = re.sub("bürgerschaftswahlprogramm 2019(.*)bürgerschaftswahlprogramm 2019bündnis 90/die grünen landesverband bremen", "", text)
    text = text.replace("präambelbürgerschaftswahlprogramm 2019bündnis 90/die grünen landesverband bremen", "")
    text = text.replace("ökologischbürgerschaftswahlprogramm 2019bündnis 90/die grünen landesverband bremenmehrnützt allen ", "")
    text = text.replace("gerechtbürgerschaftswahlprogramm 2019bündnis 90/die grünen landesverband bremenmehrnützt allen ", "")
    text = text.replace("weltoffenbürgerschaftswahlprogramm 2019bündnis 90/die grünen landesverband bremenmehrnützt allen ", "")
    text = text.replace("bündnis 90/die grünen", "wir")
    text = text.replace("europas versprechen erneuern.europawahlprogramm 2019wirpräambel", "")
    text = text.replace("europawahlprogramm 2019europawahlprogramm 2019wirpräambel", "")
    text = text.replace("europas versprechen erneuern.europawahlprogramm 2019wirerhalten, was uns erhält", "")
    text = text.replace("europawahlprogramm 2019europawahlprogramm 2019wirerhalten, was uns erhält", "")
    text = text.replace("europawahlprogramm 2019wirstärken, was uns zusammenhält", "")
    text = text.replace("europawahlprogramm 2019", "")
    text = text.replace("bayerns lebensgrundlagen erhaltenprogramm für die bayerische landtagswahl am 14. oktober 2018", "")
    text = text.replace("bayern – land der chancen für alleprogramm für die bayerische landtagswahl am 14. oktober 2018", "")
    text = text.replace("bayern – bunt, frei, sicherprogramm für die bayerische landtagswahl am 14. oktober 2018", "")
    text = text.replace("bayern und die welt – zusammenhalt macht stark programm für die bayerische landtagswahl am 14. oktober 2018", "")
    text = text.replace("programm für die bayerische landtagswahl am 14. oktober 2018bayerns lebensgrundlagen erhalten", "")
    text = text.replace("programm für die bayerische landtagswahl am 14. oktober 2018bayern – land der chancen für alle", "")
    text = text.replace("programm für die bayerische landtagswahl am 14. oktober 2018bayern – bunt, frei, sicher", "")
    text = text.replace("programm für die bayerische landtagswahl am 14. oktober 2018bayern und die welt – zusammenhalt macht stark", "")
    
    # Linke
    text = text.replace("DIE LINKE", "Wir")
    
    # SPD
    text = re.sub("Das Zukunftsprogramm der SPDKapitel\s\dSPD-Parteivorstand 2021", "", text)
    text = text.replace("die spd", "wir")
    
    # Union
    text = text.replace("CDU und CSU", "wir")
    text = re.sub("\d+\.\d+", "", text)
    
    # Remove possible newly occured double spaces
    text = re.sub("\s+", " ", text)
    
    return text

In [ ]:
def merge_paragraphs(all_paragraphs):
    tmp = []
    for p in all_paragraphs:        
        if (p[-1] in [".", "!", "?"]):
            if ((len(tmp) == 0) and (p[0].islower())):
                continue
            tmp.append(p)
            yield " ".join(tmp)
            tmp = []
        elif (p[-1] == ":"): 
            continue
        elif ((p[0].isupper()) and (tmp == [])):
            tmp.append(p)

In [ ]:
df_all_paragraphs = pd.DataFrame(columns=["text_orig", "party"])

for _, program in df_party_programs_overview.iterrows():
    all_text = extract_text(data_dir / "src" / program["party"] / f"{program['election']}.pdf", page_numbers=range(program["first_page"] - 1, program["last_page"]))

    all_paragraphs = list(filter(lambda x : 150 <= len(x), all_text.split("\n\n")))
        
    all_paragraphs = [initial_cleaning(x) for x in all_paragraphs]
    all_paragraphs = list(filter(lambda x : 150 <= len(x), all_paragraphs))
    all_paragraphs = merge_paragraphs(all_paragraphs)
    all_paragraphs = [initial_cleaning(x, lower=True) for x in all_paragraphs]

    df_current_party = pd.DataFrame(all_paragraphs, columns=["text_orig"])
    df_current_party["party"] = program["party"]
    df_current_party["election_type"] = program["election"][:3]
    df_current_party["election"] = program["election"]

    df_all_paragraphs = pd.concat([df_all_paragraphs, df_current_party])

In [ ]:
df_all_paragraphs

## 3. Data Understanding

In [ ]:
df_understanding = df_all_paragraphs.copy().reset_index(drop=True)

`Anzahl an Paragraphen pro Partei`

In [ ]:
sns.countplot(x="party", data=df_understanding)

### `Anzahl der Paragraphen pro Art der Wahl`

In [ ]:
sns.countplot(x="party", hue="election_type", data=df_understanding)

### `Häufigste 3-grams`

In [ ]:
for party in ["AfD", "FDP", "Grüne", "Linke", "SPD", "Union"]:
    df_current_party = df_understanding[df_understanding["party"] == party]
    party_text = " ".join(df_current_party["text_orig"].tolist())
    ngram_counts = Counter(ngrams(party_text.split(), 3))
    print(f"{party}: " + str(ngram_counts.most_common(5)))
    print("\n")

### `Verteilung der Anzahl an...`

In [ ]:
df_understanding["char_count"] = df_understanding["text_orig"].parallel_apply(lambda x : len(x))
df_understanding["word_count"] = df_understanding["text_orig"].parallel_apply(lambda x : len(x.split()))
df_understanding["sentence_count"] = df_understanding["text_orig"].parallel_apply(lambda x : len(sent_tokenize(x)))

#### `Zeichen`

In [ ]:
sns.histplot(df_understanding["char_count"])

#### `Wörtern`

In [ ]:
sns.histplot(df_understanding["word_count"])

#### `Sätzen`

In [ ]:
sns.histplot(df_understanding["sentence_count"])

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "oliverguhr/german-sentiment-bert"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
df_understanding["num_tokens"] = df_understanding["text_orig"].apply(lambda x: len(tokenizer.encode(x)))

In [ ]:
df_understanding["num_tokens"].describe()

## 4. Data Preparation

In [ ]:
df_prep = df_all_paragraphs.copy().reset_index(drop=True)

In [ ]:
if FAST_MODE and (data_dir / "cache/party_programs_prep.feather").exists():
    df_prep = pd.read_feather(data_dir / "cache/party_programs_prep.feather")
else:
    clean = CleanText()

    df_prep["clean_text"] = df_prep["text_orig"].parallel_apply(lambda x: clean.clean_text(x, True))
    df_prep["tokenized_text"] = df_prep["clean_text"].parallel_apply(lambda x: clean.remove_stopwords(clean.stemm_text(x)))

    if (data_dir / "cache").exists() == False:
        (data_dir / "cache").mkdir()
    df_prep.to_feather(data_dir / "cache/party_programs_prep.feather")


In [ ]:
df_prep

## 5. Modelling

In [ ]:
df_final = df_prep.copy().reset_index(drop=True).drop(columns=["text_orig", "election_type", "election"])

In [ ]:
with open("../../data/parties.json", "r") as f:
  party_encoding = json.load(f)
  df_final["party"] = df_final["party"].map(party_encoding)

In [ ]:
df_final

In [ ]:
df_final.to_parquet(data_dir / "party_programs.parquet", index=False)